In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Rescaling
from sklearn.ensemble import RandomForestClassifier
from abc import ABC, abstractmethod
import warnings

In [2]:
# suppress harmless warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [3]:
class MnistClassifierInterface(ABC):
    """
    Definition of an abstract class, that has no implementation yet.
    Defines base for derived MNIST classifiers.
    It puts a requirement for train() and predict() to be implemented further.
    """
    
    @abstractmethod
    def train(self, X_train, y_train):
        """Trains the specific model using the provided data."""
        pass

    @abstractmethod
    def predict(self, X_test):
        """Performs predictions on the test data."""
        pass

In [4]:
class RFClassifier(MnistClassifierInterface):
    """
    Random Forest Classifier implementation.
    Requires flattened image input (N, 784).
    """
    def __init__(self):
        N_ESTIMATORS = 10 # forest tree count
        N_JOBS = -1 # use all cores available
        self.model = RandomForestClassifier(random_state=42, n_estimators=N_ESTIMATORS, n_jobs=N_JOBS)
        print("RF Classifier initialized.")

    def train(self, X_train, y_train):
        print("Starting RF training...")
        # random Forest requires flattened input, which is handled in the MnistClassifier setup
        self.model.fit(X_train, y_train)
        print("RF training complete.")

    def predict(self, X_test):
        # returns the predicted class (0-9)
        return self.model.predict(X_test)

In [5]:
class NNClassifier(MnistClassifierInterface):
    """
    Feed-Forward Neural Network (Dense layers) implementation.
    Requires flattened and normalized input (N, 784).
    Assembled of dense layers with neuron quantity of 2^n.
    """
    def __init__(self):
        # input shape expects the flattened 784 features
        N_CLASSES = 10 # 10 classes (digits 0-9)
        PIXEL_TOTAL = 28 * 28 # total pixels
        LAYER1_NEURONS = 128
        LAYER2_NEURONS = 64
        self.model = Sequential([
            Dense(LAYER1_NEURONS, activation='relu', input_shape=(PIXEL_TOTAL,)),
            Dense(LAYER2_NEURONS, activation='relu'),
            Dense(N_CLASSES, activation='softmax')
        ])
        self.model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy', # works with integer labels (0-9)
            metrics=['accuracy']
        )
        print("NN Classifier initialized.")

    def train(self, X_train, y_train):
        """
        NN requires normalized input (0-1),
        which is handled in the MnistClassifier setup
        """
        
        print("Starting NN training (1 epoch)...")
        N_EPOCHS = 1 # training iterations
        BATCH_SIZE = 32
        IS_VERBOSE = False # suppress NN logs
        self.model.fit(X_train, y_train, epochs=N_EPOCHS, batch_size=BATCH_SIZE, verbose=IS_VERBOSE)
        print("NN training complete.")

    def predict(self, X_test):
        # Returns probabilities; we need to convert to class index (0-9)
        IS_VERBOSE = False # suppress NN logs
        probabilities = self.model.predict(X_test, verbose=IS_VERBOSE)
        return np.argmax(probabilities, axis=1)

In [6]:
class CNNClassifier(MnistClassifierInterface):
    """
    Convolutional Neural Network implementation for MNIST.
    Requires 4D input (N, 28, 28, 1) and normalization (0-1).
    """
    def __init__(self):
        self.model = Sequential([
            # input shape is 28x28x1
            Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            Flatten(),
            Dense(10, activation='softmax')
        ])
        self.model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )
        print("CNN Classifier initialized.")

    def train(self, X_train, y_train):
        print("Starting CNN training (1 epoch)...")
        # CNN requires 4D input and normalization, handled in MnistClassifier setup
        self.model.fit(X_train, y_train, epochs=1, batch_size=32, verbose=0)
        print("CNN training complete.")

    def predict(self, X_test):
        # returns probabilities; we need to convert to class index (0-9)
        probabilities = self.model.predict(X_test, verbose=0)
        return np.argmax(probabilities, axis=1)

In [7]:
# factory-type class

class MnistClassifier:
    """
    A unified interface that selects the correct concrete classifier 
    and handles data preprocessing specific to the chosen algorithm.
    """
    
    ALGORITHMS = {'rf': RFClassifier, 'nn': NNClassifier, 'cnn': CNNClassifier}

    def __init__(self, algorithm: str, X_data, y_data):
        """
        Initializes the MnistClassifier by selecting the correct concrete model 
        and preparing the data for that model.
        """
        algorithm = algorithm.lower()
        if algorithm not in self.ALGORITHMS:
            raise ValueError(f"Unknown algorithm: {algorithm}. Must be one of {list(self.ALGORITHMS.keys())}")

        self.algorithm_name = algorithm
        self.model: MnistClassifierInterface = self.ALGORITHMS[algorithm]()
        
        # store original data
        self._X_train_orig, self._y_train = X_data[0], y_data[0]
        self._X_test_orig, self._y_test = X_data[1], y_data[1]

        # process data based on algorithm requirements
        self.X_train, self.X_test = self._preprocess_data(algorithm)

    def _preprocess_data(self, algorithm):
        """Handles necessary reshaping and normalization based on the model type."""
        
        # cast to float32 for TensorFlow/NumPy operations
        X_train, X_test = self._X_train_orig.astype('float32'), self._X_test_orig.astype('float32')

        if algorithm in ['rf', 'nn']:
            # RF and Feed-Forward NN require flattened input (N, 784)
            print(f"Preprocessing: Flattening data for {algorithm.upper()}...")
            N_train, H, W = X_train.shape
            N_test, _, _ = X_test.shape
            
            # reshape using -1 to calculate the features automatically
            X_train_processed = X_train.reshape(N_train, -1)
            X_test_processed = X_test.reshape(N_test, -1)

            # NN requires normalization for consistency
            if algorithm == 'nn':
                print("Preprocessing: Normalizing data for NN...")
                X_train_processed /= 255.0
                X_test_processed /= 255.0
            
            return X_train_processed, X_test_processed

        elif algorithm == 'cnn':
            # CNN requires 4D input (N, 28, 28, 1) and normalization (0-1)
            print("Preprocessing: Reshaping and normalizing data for CNN...")
            # reshape to 4D: N, H, W, Channels
            X_train_processed = X_train[..., np.newaxis]
            X_test_processed = X_test[..., np.newaxis]
            
            # normalize pixel values
            X_train_processed /= 255.0
            X_test_processed /= 255.0
            
            return X_train_processed, X_test_processed

    def train(self):
        """Delegates the training call to the underlying model."""
        self.model.train(self.X_train, self._y_train)

    def predict(self):
        """
        Delegates the prediction call and returns the result, 
        maintaining a consistent output structure regardless of model.
        """
        print(f"\nMaking predictions using {self.algorithm_name.upper()}...")
        return self.model.predict(self.X_test)
    
    def evaluate(self, y_pred):
        """Calculates and prints the accuracy of the predictions."""
        from sklearn.metrics import accuracy_score
        accuracy = accuracy_score(self._y_test, y_pred)
        print(f"Accuracy for {self.algorithm_name.upper()}: {accuracy:.4f}")
        return accuracy

In [8]:
# data loading

def main():
    print("Data loading and preparation...")
    (X_train, y_train), (X_test, y_test) = mnist.load_data()

    # use only a small subset of data for fast demonstration
    subset_size = 5000
    X_train, y_train = X_train[:subset_size], y_train[:subset_size]
    X_test, y_test = X_test[:subset_size], y_test[:subset_size]
    print(f"Loaded {len(X_train)} training samples and {len(X_test)} test samples.")
    data_X = (X_train, X_test)
    data_y = (y_train, y_test)

    # List of algorithms to test
    algorithms_to_test = ['rf', 'nn', 'cnn']
    results = {}
    for algo in algorithms_to_test:
        print(f"\n==========================================")
        print(f"STARTING MODEL: {algo.upper()}")
        print(f"==========================================")
        
        try:
            # classifier instance
            classifier = MnistClassifier(algo, data_X, data_y)
            
            # model training
            classifier.train()
            
            # test prediction
            predictions = classifier.predict()
            
            # performance estimation
            accuracy = classifier.evaluate(predictions)
            results[algo] = accuracy
            
            # head elements among predicted classes
            print(f"First 5 predictions: {predictions[:5].tolist()}")

        except Exception as e:
            print(f"An error occurred with {algo.upper()}: {e}")
            results[algo] = "Failed"

    print("\n--- Summary of Results ---")
    for algo, acc in results.items():
        print(f"| {algo.upper():<3} Accuracy: {acc if isinstance(acc, str) else f'{acc:.4f}'}")


if __name__ == "__main__":
    main()


Data loading and preparation...
Loaded 5000 training samples and 5000 test samples.

STARTING MODEL: RF
RF Classifier initialized.
Preprocessing: Flattening data for RF...
Starting RF training...
RF training complete.

Making predictions using RF...
Accuracy for RF: 0.8498
First 5 predictions: [7, 2, 1, 0, 4]

STARTING MODEL: NN
NN Classifier initialized.
Preprocessing: Flattening data for NN...
Preprocessing: Normalizing data for NN...
Starting NN training (1 epoch)...
NN training complete.

Making predictions using NN...
Accuracy for NN: 0.8550
First 5 predictions: [7, 2, 1, 0, 4]

STARTING MODEL: CNN
CNN Classifier initialized.
Preprocessing: Reshaping and normalizing data for CNN...
Starting CNN training (1 epoch)...
CNN training complete.

Making predictions using CNN...
Accuracy for CNN: 0.9094
First 5 predictions: [7, 2, 1, 0, 4]

--- Summary of Results ---
| RF  Accuracy: 0.8498
| NN  Accuracy: 0.8550
| CNN Accuracy: 0.9094
